<a href="https://colab.research.google.com/github/Panperception/QKD/blob/main/QKD_AI3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required libs

In [ ]:
# %pip uninstall qiskit
!pip install pyqmc
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit-algorithms
!pip install qiskit-nature
!pip install qutip
!pip install ase
!pip install scipy
!nvcc --version

## A whole set of tests

Below is a single, self‐contained Python script that integrates data preparation for a shared synthetic dataset and runs experiments for several advanced QKD–AI scenarios. In this unified example, we generate synthetic data for multiple tasks (error correction parameter estimation, quantum feedback control, decoy‐state optimization, and multi‐modal attack detection) and also include experiments for a reinforcement learning (RL)–based key rate optimizer and a genetic algorithm for end‐to‐end protocol parameter optimization.



In [2]:
!pip install stable_baselines3
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D, Concatenate
from sklearn.model_selection import train_test_split
import random
import gym
from gym import spaces
from stable_baselines3 import DQN
from deap import base, creator, tools, algorithms

# -------------------------
# 0. Set Random Seeds
# -------------------------
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# -------------------------
# 1. Data Preparation
# -------------------------
# We define several synthetic datasets to simulate different aspects of a QKD system.

def create_error_correction_dataset(num_samples=1000):
    # Features: [channel_loss, noise, QBER, misc_stat]
    X = np.random.rand(num_samples, 4)
    # Label: optimal error correction rate (higher when QBER is low)
    y = 1 - X[:, 2] + 0.05 * np.random.randn(num_samples)
    return X, y

def create_qber_series_dataset(num_series=500, timesteps=20):
    # Each sample is a time series representing measured QBER over time.
    X = []
    y = []
    for _ in range(num_series):
        base = np.random.rand() * 0.1  # baseline QBER
        noise = np.random.randn(timesteps) * 0.01
        series = base + noise
        # Control signal: corrective value to bring the last measurement toward target (e.g., 0.02)
        target = 0.02
        control = -(series[-1] - target)
        X.append(series.reshape(timesteps, 1))
        y.append(control)
    return np.array(X), np.array(y)

def create_decoy_dataset(num_samples=1000):
    # Features: [channel_loss, noise_level]
    X = np.random.rand(num_samples, 2)
    # Label: optimal decoy intensity (for simulation, we assume an approximately linear relation)
    y = 0.5 + 0.2*(X[:, 0] - 0.5) + 0.05*np.random.randn(num_samples)
    return X, y

def create_multimodal_dataset(num_samples=1000):
    # Create three modalities:
    #  - QBER features (5 values)
    #  - Detector noise stats (3 values)
    #  - Timing jitter (2 values)
    X_qber = np.random.rand(num_samples, 5)
    X_noise = np.random.rand(num_samples, 3)
    X_jitter = np.random.rand(num_samples, 2)
    # Binary label: 0 (normal) or 1 (attack)
    y = np.random.randint(0, 2, size=(num_samples, 1))
    return (X_qber, X_noise, X_jitter), y

# Generate datasets
X_ec, y_ec = create_error_correction_dataset()
X_qber_series, y_qber_series = create_qber_series_dataset()
X_decoy, y_decoy = create_decoy_dataset()
(X_mm_qber, X_mm_noise, X_mm_jitter), y_mm = create_multimodal_dataset()

# Split datasets for supervised tasks (error correction, decoy, multimodal detection)
X_ec_train, X_ec_test, y_ec_train, y_ec_test = train_test_split(X_ec, y_ec, test_size=0.2, random_state=42)
X_decoy_train, X_decoy_test, y_decoy_train, y_decoy_test = train_test_split(X_decoy, y_decoy, test_size=0.2, random_state=42)
X_mm_qber_train, X_mm_qber_test, y_mm_train = train_test_split(X_mm_qber, y_mm, test_size=0.2, random_state=42)
X_mm_noise_train, X_mm_noise_test, _ = train_test_split(X_mm_noise, y_mm, test_size=0.2, random_state=42)
X_mm_jitter_train, X_mm_jitter_test, _ = train_test_split(X_mm_jitter, y_mm, test_size=0.2, random_state=42)

# -------------------------
# 2. Experiment Modules
# -------------------------

# 2.1. RL: Real-Time Dynamic Key Rate Optimization Environment
class QKDEnv(gym.Env):
    def __init__(self):
        super(QKDEnv, self).__init__()
        # State: [channel_loss, noise_level] between 0 and 1
        self.observation_space = spaces.Box(low=0, high=1, shape=(2,), dtype=np.float32)
        # Action: modulation parameter in [0,1]
        self.action_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
        self.state = np.array([0.5, 0.5])
        self.step_count = 0

    def step(self, action):
        channel_loss, noise = self.state
        modulation = action[0]
        # Simulated QBER: lower if modulation is higher
        qber = noise * (1 - modulation) + np.random.rand()*0.01
        # Key rate: ideally higher when modulation is high and QBER is low
        key_rate = modulation * (1 - qber)
        reward = key_rate
        # Update state with small random fluctuation
        self.state = np.clip(self.state + np.random.randn(2)*0.01, 0, 1)
        self.step_count += 1
        done = self.step_count > 100
        return self.state, reward, done, {}

    def reset(self):
        self.state = np.array([0.5, 0.5])
        self.step_count = 0
        return self.state

# 2.2. Adaptive Error Correction Parameter Estimation Model
def build_error_correction_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# 2.3. Quantum Feedback Control Using LSTM Model
def build_feedback_model(timesteps, features=1):
    model = Sequential([
        LSTM(32, input_shape=(timesteps, features)),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# 2.4. Adaptive Decoy-State Protocol Predictor
def build_decoy_model():
    model = Sequential([
        Dense(32, activation='relu', input_shape=(2,)),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# 2.5. Multi-Modal Attack Detection Model
def build_multimodal_model():
    input_qber = Input(shape=(5,))
    input_noise = Input(shape=(3,))
    input_jitter = Input(shape=(2,))

    x1 = Dense(16, activation='relu')(input_qber)
    x2 = Dense(16, activation='relu')(input_noise)
    x3 = Dense(16, activation='relu')(input_jitter)

    combined = Concatenate()([x1, x2, x3])
    x = Dense(32, activation='relu')(combined)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[input_qber, input_noise, input_jitter], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 2.6. End-to-End QKD Protocol Design via Genetic Algorithm
def optimize_protocol_parameters():
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()
    # Parameters: [modulation_index, decoy_intensity, error_corr_rate]
    toolbox.register("attr_float", random.uniform, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 3)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    def evaluate_protocol(individual):
        modulation_index, decoy_intensity, error_corr_rate = individual
        key_rate = modulation_index * (1 - abs(decoy_intensity - 0.5)) * error_corr_rate
        return (key_rate,)
    toolbox.register("evaluate", evaluate_protocol)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    population = toolbox.population(n=50)
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, verbose=False)
    best_ind = tools.selBest(population, 1)[0]
    return best_ind, evaluate_protocol(best_ind)[0]

# -------------------------
# 3. Running All Experiments
# -------------------------
if __name__ == "__main__":
    # Experiment 1: RL for Dynamic Key Rate Optimization
    print("=== RL: Real-Time Dynamic Key Rate Optimization ===")
    env = QKDEnv()
    rl_model = DQN('MlpPolicy', env, verbose=0)
    rl_model.learn(total_timesteps=5000)
    state = env.reset()
    for i in range(10):
        action, _ = rl_model.predict(state)
        state, reward, done, _ = env.step(action)
        print(f"Step {i+1}: State = {state}, Reward = {reward:.4f}")
        if done:
            break

    # Experiment 2: Adaptive Error Correction Parameter Estimation
    print("\n=== Adaptive Error Correction Parameter Estimation ===")
    ec_model = build_error_correction_model()
    ec_model.fit(X_ec_train, y_ec_train, epochs=30, batch_size=32, validation_data=(X_ec_test, y_ec_test), verbose=0)
    ec_loss, ec_mae = ec_model.evaluate(X_ec_test, y_ec_test, verbose=0)
    print("Error Correction Model Test MAE:", ec_mae)

    # Experiment 3: Quantum Feedback Control with LSTM
    print("\n=== Quantum Feedback Control (LSTM) ===")
    timesteps = X_qber_series.shape[1]
    feedback_model = build_feedback_model(timesteps, features=1)
    feedback_model.fit(X_qber_series, y_qber_series, epochs=20, batch_size=32, validation_split=0.2, verbose=0)
    fb_loss = feedback_model.evaluate(X_qber_series, y_qber_series, verbose=0)
    print("Feedback Model Loss:", fb_loss)

    # Experiment 4: Adaptive Decoy-State Protocol Optimization Predictor
    print("\n=== Adaptive Decoy-State Protocol Optimization ===")
    decoy_model = build_decoy_model()
    decoy_model.fit(X_decoy_train, y_decoy_train, epochs=30, batch_size=32, validation_data=(X_decoy_test, y_decoy_test), verbose=0)
    decoy_loss = decoy_model.evaluate(X_decoy_test, y_decoy_test, verbose=0)
    print("Decoy Model Loss:", decoy_loss)

    # Experiment 5: Multi-Modal Attack Detection
    print("\n=== Multi-Modal Attack Detection ===")
    mm_model = build_multimodal_model()
    mm_model.fit([X_mm_qber_train, X_mm_noise_train, X_mm_jitter_train], y_mm_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)
    mm_loss, mm_acc = mm_model.evaluate([X_mm_qber_test, X_mm_noise_test, X_mm_jitter_test], y_mm_test, verbose=0)
    print("Multi-Modal Model Test Accuracy:", mm_acc)

    # Experiment 6: End-to-End QKD Protocol Design via Genetic Algorithm
    print("\n=== End-to-End QKD Protocol Design via Genetic Algorithm ===")
    best_protocol, best_key_rate = optimize_protocol_parameters()
    print("Best protocol parameters:", best_protocol)
    print("Achieved simulated key rate:", best_key_rate)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

ModuleNotFoundError: No module named 'deap'

## A more sophisticated test
Below is an example of a more comprehensive simulation framework that integrates a more realistic QKD‐system model with several AI modules. In this script, we simulate a decoy‐state QKD system using a simplified—but more realistic—model based on Poissonian photon statistics, channel loss, dark counts, and noise. Then, we use the simulation to generate sophisticated data that feed into several experiments: an RL‐based key rate optimizer, an adaptive error‐correction estimator, an LSTM-based feedback controller, a decoy-state predictor, a multi‐modal attack detection module, and a genetic algorithm for protocol parameter optimization.

**Note: This simulation is still a simplified “toy” model for illustration. In practice you may need to replace or extend the functions (e.g. the QBER and key rate calculations) with models derived from your experimental QKD system.**

In [9]:
!pip install deap # Install the missing 'deap' library.
!pip install 'shimmy>=2.0' # Install shimmy to support OpenAI Gym environments with SB3

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D, Concatenate
from sklearn.model_selection import train_test_split
import random
import gym
from gym import spaces
from stable_baselines3 import DQN
from deap import base, creator, tools, algorithms
import math

# -------------------------
# 0. Set Random Seeds
# -------------------------
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# -------------------------
# 1. Realistic QKD Simulation Functions
# -------------------------
def poisson_photon_number(mu):
    """Simulate photon number from a Poisson distribution with mean mu."""
    return np.random.poisson(mu)

def simulate_qkd_run(mu, t, dark_count=1e-5, noise_std=0.01, attack=False):
    """
    Simulate one round of decoy-state QKD.

    Parameters:
      mu: average photon number (signal intensity)
      t: channel transmittance (0 to 1)
      dark_count: dark count probability per detector
      noise_std: standard deviation of noise in detection
      attack: if True, simulate an attack (e.g., intercept-resend increases errors)

    Returns:
      qber: Quantum Bit Error Rate (float between 0 and 1)
      key_rate: secure key rate (a simplified function)
      decoy_state: simulated decoy measurement (could be the observed intensity)
    """
    # Simulate the number of photons sent in a pulse
    n_photons = poisson_photon_number(mu)
    # Photons transmitted through channel: each photon survives with probability t
    n_detected = np.sum(np.random.rand(n_photons) < t)

    # Dark counts: simulate extra counts (assuming a fixed probability per pulse)
    dark = 1 if np.random.rand() < dark_count else 0

    # The total counts at Bob
    total_counts = n_detected + dark
    # Base error probability due to noise
    base_error = noise_std
    # If an attacker is present, errors increase (e.g., by 0.05)
    attack_error = 0.05 if attack else 0.0

    # QBER: if total_counts > 0, errors occur with probability base_error+attack_error
    if total_counts > 0:
        errors = np.random.binomial(total_counts, base_error + attack_error)
        qber = errors / total_counts
    else:
        qber = 0.5  # if no detection, assume maximum uncertainty

    # Secure key rate estimation using a simple model: key_rate = total_counts*(1 - H(qber))
    # H(x): binary entropy function.
    def binary_entropy(x):
        if x==0 or x==1:
            return 0.0
        return -x*math.log2(x) - (1-x)*math.log2(1-x)

    key_rate = total_counts * (1 - binary_entropy(qber))
    # Simulate decoy state measurement as observed intensity with noise
    decoy_state = mu + np.random.randn()*0.05
    return qber, key_rate, decoy_state

def generate_qkd_dataset(num_samples=1000, attack_ratio=0.2):
    """
    Generate a dataset for supervised learning tasks.
    Each sample includes channel parameters and simulation outputs.

    Returns:
      X: features matrix including [mu, t, dark_count, noise_std, decoy_state, attack_flag]
      y_qber: simulated QBER
      y_key_rate: simulated key rate
    """
    X = []
    y_qber = []
    y_key_rate = []
    for _ in range(num_samples):
        # Randomly choose parameters in realistic ranges:
        mu = np.random.uniform(0.1, 1.0)           # signal intensity
        t = np.random.uniform(0.1, 0.9)              # channel transmittance
        dark = 1e-5                                  # dark count (fixed for now)
        noise_std = np.random.uniform(0.005, 0.02)     # noise level
        attack_flag = 1 if np.random.rand() < attack_ratio else 0
        qber, key_rate, decoy_state = simulate_qkd_run(mu, t, dark, noise_std, attack=bool(attack_flag))
        features = [mu, t, dark, noise_std, decoy_state, attack_flag]
        X.append(features)
        y_qber.append(qber)
        y_key_rate.append(key_rate)
    return np.array(X), np.array(y_qber), np.array(y_key_rate)

# Generate a shared QKD dataset
X_qkd, y_qkd, key_rates = generate_qkd_dataset(num_samples=2000, attack_ratio=0.3)
# For some supervised tasks, we can use different target variables:
#  - For error correction estimation, we might use y_qkd (QBER)
#  - For decoy-state optimization, we might use decoy_state as part of X and key_rates

# Split the QKD dataset (we'll use it for training some of our modules)
X_qkd_train, X_qkd_test, y_qkd_train, y_qkd_test = train_test_split(X_qkd, y_qkd, test_size=0.2, random_state=42)

# -------------------------
# 2. Experiment Modules (Refined with QKD Simulation Data)
# -------------------------

# 2.1. RL Environment for Dynamic Key Rate Optimization
class QKDEnvRealistic(gym.Env):
    def __init__(self):
        super(QKDEnvRealistic, self).__init__()
        # State: [mu, t, noise_std] - basic channel parameters
        self.observation_space = spaces.Box(low=0, high=1, shape=(3,), dtype=np.float32)
        # Action: modulation adjustment in [0,1] that might affect key rate
        self.action_space = spaces.Discrete(10)
        # Initialize state with random parameters
        self.state = np.array([0.5, 0.5, 0.01])
        self.step_count = 0

    def step(self, action):
        mu, t, noise = self.state
        modulation = action / (self.action_space.n - 1)
        # We assume modulation improves effective transmittance:
        effective_t = np.clip(t * modulation, 0, 1)
        # Simulate one round of QKD with these parameters (no attack in RL env)
        qber, key_rate, _ = simulate_qkd_run(mu, effective_t, dark_count=1e-5, noise_std=noise, attack=False)
        # Reward: key_rate normalized by mu for example
        reward = key_rate / mu
        # Update state with slight fluctuation (simulate slowly varying channel)
        self.state = np.clip(self.state + np.random.randn(3)*0.005, 0, 1)
        self.step_count += 1
        done = self.step_count > 100
        return self.state, reward, done, {}

    def reset(self):
        self.state = np.array([0.5, 0.5, 0.01])
        self.step_count = 0
        return self.state

# 2.2. Adaptive Error Correction Parameter Estimation Model
# Here we use the QKD simulation dataset: features: [mu, t, dark, noise, decoy, attack_flag]
# and target: y_qkd (QBER)
def build_error_correction_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_qkd_train.shape[1],)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# 2.3. Quantum Feedback Control Using LSTM
# We simulate a time series by generating a sequence of QBER values from consecutive QKD runs.
def generate_qber_series_from_simulation(num_series=500, timesteps=20):
    X_series = []
    y_feedback = []
    for _ in range(num_series):
        series = []
        # Start with a random state
        mu = np.random.uniform(0.2, 0.8)
        t = np.random.uniform(0.3, 0.9)
        noise = np.random.uniform(0.005, 0.02)
        for _ in range(timesteps):
            qber, _, _ = simulate_qkd_run(mu, t, dark_count=1e-5, noise_std=noise, attack=False)
            series.append(qber)
            # Let channel parameters drift slowly
            mu = np.clip(mu + np.random.randn()*0.01, 0.2, 0.8)
            t = np.clip(t + np.random.randn()*0.01, 0.3, 0.9)
        series = np.array(series)
        # Desired control: bring last QBER to a target (e.g., 0.02)
        target = 0.02
        control = -(series[-1] - target)
        X_series.append(series.reshape(timesteps, 1))
        y_feedback.append(control)
    return np.array(X_series), np.array(y_feedback)

X_qber_series, y_feedback = generate_qber_series_from_simulation()

def build_feedback_model(timesteps, features=1):
    model = Sequential([
        LSTM(32, input_shape=(timesteps, features)),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# 2.4. Adaptive Decoy-State Protocol Predictor
# Using our QKD dataset, we use the decoy measurement (column index 4) as part of features.
# Here, we try to predict the optimal decoy intensity given channel loss and noise.
def build_decoy_model():
    # Input: [t, noise] from our QKD features (columns 1 and 3)
    model = Sequential([
        Dense(32, activation='relu', input_shape=(2,)),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# 2.5. Multi-Modal Attack Detection Model
# For this example, we simulate multi-modal data from the QKD simulation.
# We create three modalities:
#   - QBER series (5 values): simulated over 5 runs
#   - Detector noise: random vector of 3 values
#   - Timing jitter: random vector of 2 values
def create_multimodal_qkd_dataset(num_samples=1000):
    X_qber = np.random.rand(num_samples, 5) * 0.1  # QBER values around 0.0-0.1
    X_noise = np.random.rand(num_samples, 3) * 0.02  # noise levels
    X_jitter = np.random.rand(num_samples, 2) * 0.005  # jitter in seconds
    # Label: simulate attack if average QBER > threshold or high noise (simple rule)
    y = ((X_qber.mean(axis=1) > 0.05) | (X_noise.mean(axis=1) > 0.015)).astype(int).reshape(-1, 1)
    return (X_qber, X_noise, X_jitter), y

(X_mm_qber, X_mm_noise, X_mm_jitter), y_mm = create_multimodal_qkd_dataset()
X_mm_qber_train, X_mm_qber_test, y_mm_train, y_mm_test = train_test_split(X_mm_qber, y_mm, test_size=0.2, random_state=42)
# Added y_mm_test to capture the fourth return value from train_test_split
X_mm_noise_train, X_mm_noise_test, y_mm_noise_train, y_mm_noise_test_ = train_test_split(X_mm_noise, y_mm, test_size=0.2, random_state=42)
X_mm_jitter_train, X_mm_jitter_test, y_mm_jitter_train, y_mm_jitter_test = train_test_split(X_mm_jitter, y_mm, test_size=0.2, random_state=42)

def build_multimodal_model():
    input_qber = Input(shape=(5,))
    input_noise = Input(shape=(3,))
    input_jitter = Input(shape=(2,))

    x1 = Dense(16, activation='relu')(input_qber)
    x2 = Dense(16, activation='relu')(input_noise)
    x3 = Dense(16, activation='relu')(input_jitter)

    combined = Concatenate()([x1, x2, x3])
    x = Dense(32, activation='relu')(combined)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[input_qber, input_noise, input_jitter], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 2.6. End-to-End QKD Protocol Design via Genetic Algorithm
def optimize_protocol_parameters():
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()
    # Parameters: [modulation_index, decoy_intensity, error_corr_rate]
    toolbox.register("attr_float", random.uniform, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 3)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    def evaluate_protocol(individual):
        modulation_index, decoy_intensity, error_corr_rate = individual
        # Use a more realistic key rate function: assume key_rate depends on channel transmittance and QBER.
        # Here we use a simulated formula: key_rate = modulation_index * (1 - abs(decoy_intensity - 0.5)) * error_corr_rate
        return (modulation_index * (1 - abs(decoy_intensity - 0.5)) * error_corr_rate,)
    toolbox.register("evaluate", evaluate_protocol)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    population = toolbox.population(n=50)
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, verbose=False)
    best_ind = tools.selBest(population, 1)[0]
    return best_ind, evaluate_protocol(best_ind)[0]

# -------------------------
# 3. Running the Complete Simulation Framework
# -------------------------
if __name__ == "__main__":
    # Experiment 1: RL for Dynamic Key Rate Optimization (Realistic QKD Env)
    print("=== RL: Real-Time Dynamic Key Rate Optimization (Realistic) ===")
    env = QKDEnvRealistic()
    rl_model = DQN('MlpPolicy', env, verbose=0)
    rl_model.learn(total_timesteps=5000)
    state = env.reset()
    for i in range(10):
        action, _ = rl_model.predict(state)
        state, reward, done, _ = env.step(action)
        print(f"Step {i+1}: State = {state}, Reward = {reward:.4f}")
        if done:
            break

    # Experiment 2: Adaptive Error Correction Parameter Estimation (Predict QBER)
    print("\n=== Adaptive Error Correction Parameter Estimation ===")
    ec_model = build_error_correction_model()
    ec_model.fit(X_qkd_train, y_qkd_train, epochs=30, batch_size=32, validation_split=0.2, verbose=0)
    ec_loss, ec_mae = ec_model.evaluate(X_qkd_test, y_qkd_test, verbose=0)
    print("Error Correction Model Test MAE (QBER prediction):", ec_mae)

    # Experiment 3: Quantum Feedback Control with LSTM
    print("\n=== Quantum Feedback Control (LSTM) ===")
    timesteps = X_qber_series.shape[1]
    feedback_model = build_feedback_model(timesteps, features=1)
    feedback_model.fit(X_qber_series, y_feedback, epochs=20, batch_size=32, validation_split=0.2, verbose=0)
    fb_loss = feedback_model.evaluate(X_qber_series, y_feedback, verbose=0)
    print("Feedback Model Loss:", fb_loss)

    # Experiment 4: Adaptive Decoy-State Protocol Optimization Predictor
    print("\n=== Adaptive Decoy-State Protocol Optimization ===")
    # For decoy model, use features: channel transmittance and noise (columns 1 and 3 from X_qkd)
    X_decoy_features = X_qkd[:, [1, 3]]
    X_decoy_train, X_decoy_test, y_decoy_train, y_decoy_test = train_test_split(X_decoy_features, X_qkd[:, 4], test_size=0.2, random_state=42)
    decoy_model = build_decoy_model()
    decoy_model.fit(X_decoy_train, y_decoy_train, epochs=30, batch_size=32, validation_data=(X_decoy_test, y_decoy_test), verbose=0)
    decoy_loss = decoy_model.evaluate(X_decoy_test, y_decoy_test, verbose=0)
    print("Decoy Model Loss:", decoy_loss)

    # Experiment 5: Multi-Modal Attack Detection
    print("\n=== Multi-Modal Attack Detection ===")
    mm_model = build_multimodal_model()
    mm_model.fit([X_mm_qber_train, X_mm_noise_train, X_mm_jitter_train], y_mm_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)
    mm_loss, mm_acc = mm_model.evaluate([X_mm_qber_test, X_mm_noise_test, X_mm_jitter_test], y_mm_test, verbose=0)
    print("Multi-Modal Model Test Accuracy:", mm_acc)

    # Experiment 6: End-to-End QKD Protocol Design via Genetic Algorithm
    print("\n=== End-to-End QKD Protocol Design via Genetic Algorithm ===")
    best_protocol, best_key_rate = optimize_protocol_parameters()
    print("Best protocol parameters:", best_protocol)
    print("Achieved simulated key rate:", best_key_rate)



=== RL: Real-Time Dynamic Key Rate Optimization (Realistic) ===


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Step 1: State = [0.50288996 0.49922139 0.00881875], Reward = 2.0000
Step 2: State = [0.49744173 0.498462   0.01017799], Reward = 1.9885
Step 3: State = [0.50382804 0.50063855 0.01086841], Reward = 0.0000
Step 4: State = [0.51278965 0.50023901 0.01666742], Reward = 0.0000
Step 5: State = [0.51270177 0.4913049  0.01345244], Reward = 0.0000
Step 6: State = [0.52233217 0.49112127 0.01951832], Reward = 0.0000
Step 7: State = [0.52526621 0.48565861 0.0174828 ], Reward = 0.0000
Step 8: State = [0.52560537 0.49296236 0.01649681], Reward = 0.0000
Step 9: State = [0.52966161 0.49379049 0.01376418], Reward = 0.0000
Step 10: State = [0.53399081 0.48391635 0.01098973], Reward = 1.8880

=== Adaptive Error Correction Parameter Estimation ===


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Error Correction Model Test MAE (QBER prediction): 0.14839009940624237

=== Quantum Feedback Control (LSTM) ===


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Feedback Model Loss: 6.15660464973189e-05

=== Adaptive Decoy-State Protocol Optimization ===
Decoy Model Loss: 0.07091071456670761

=== Multi-Modal Attack Detection ===
Multi-Modal Model Test Accuracy: 0.9350000023841858

=== End-to-End QKD Protocol Design via Genetic Algorithm ===
Best protocol parameters: [1.394823357023787, 0.5315929434425318, 1.7839053948323698]
Achieved simulated key rate: 2.409622309790183
